## **RADI608: Data Mining and Machine Learning**

### Assignment: APriori Algorithm
**Romen Samuel Rodis Wabina** <br>
Student, PhD Data Science in Healthcare and Clinical Informatics <br>
Clinical Epidemiology and Biostatistics, Faculty of Medicine (Ramathibodi Hospital) <br>
Mahidol University

Note: In case of Python Markdown errors, you may access the assignment through this GitHub [Link](https://github.com/rrwabina/RADI608/tree/main/Submitted)

In [181]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings('ignore')

import mlxtend
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder

from sklearn.preprocessing import LabelEncoder
import warnings 
warnings.filterwarnings('ignore')

### Perform an Apriorialgorithm to generate the association rules by using <code>support = 0.001</code>, <code>confidence = 0.5</code>, and find top 10 of the <code>RHS = “OMPZ” </code>

In [447]:
df = pd.read_csv('../data/prescriptionDB.csv')
df.set_index('Item ID')
df.fillna('', inplace = True)
df = df[['Code.1', 'Code.2']]
dataset = df.to_numpy()

encoder = TransactionEncoder()
encoder_array = encoder.fit(dataset).transform(dataset)
df_encode = pd.DataFrame(encoder_array, columns = encoder.columns_)

def encode_units(x):
    if x is False:
        return 0
    if x is True:
        return 1

frequent_itemsets = apriori(df_encode, min_support = 0.001, use_colnames = True)
rules = association_rules(frequent_itemsets, metric = 'lift', min_threshold = 1)
rule_ompz = rules[rules['consequents'] == frozenset({'OMPZ'})]

rule_ompz = rule_ompz[rule_ompz['confidence'] >= 0.50]
rule_ompz.sort_values(by=['lift'], ascending = False).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
59,(NAPX),(OMPZ),0.113511,0.740864,0.101883,0.897561,1.211506,0.017787,2.529663
23,(ASPT),(OMPZ),0.047619,0.740864,0.042082,0.883721,1.192825,0.006803,2.228571
49,(MELO),(OMPZ),0.034330,0.740864,0.029070,0.846774,1.142955,0.003636,1.691205
47,(INDM),(OMPZ),0.008306,0.740864,0.006921,0.833333,1.124813,0.000768,1.554817
45,(IBUP),(OMPZ),0.039037,0.740864,0.030731,0.787234,1.062589,0.001810,1.217940
65,(VOLS),(OMPZ),0.007752,0.740864,0.006091,0.785714,1.060538,0.000348,1.209302
53,(MOBC),(OMPZ),0.020487,0.740864,0.015504,0.756757,1.021452,0.000326,1.065338
21,(ASA.),(OMPZ),0.284884,0.740864,0.212071,0.744412,1.004789,0.001011,1.013883


### Perform an Apriori Algorithm to generate the association rules by selecting the top 20 rules at <code>support = 0.0001</code> and the LHS has at least two drugs in the basket.

In [446]:
df = pd.read_csv('../data/prescriptionDB.csv')
df.set_index('Item ID')
df.fillna('', inplace = True)
df = df[['Code.1', 'Code.2', 'Code.3', 'Code.4']]
dataset = df.to_numpy()

encoder = TransactionEncoder()
encoder_array = encoder.fit(dataset).transform(dataset)
df_encode = pd.DataFrame(encoder_array, columns = encoder.columns_)

def cariIndex(setList, data2):
   i=-1
   for data in setList:
       i=i+1
       if data==data2:
           return i
   return -1 

frequent_itemsets = apriori(df_encode, min_support = 0.0001, use_colnames = True)
rules = association_rules(frequent_itemsets, metric = 'lift', min_threshold = 5)

index1 = []
for idx, value in enumerate(rules['antecedents']):
    if len(value) >= 2:
        if cariIndex(value, '') != 0:
            index1.append(idx)
index2= []
for idx, value in enumerate(rules['consequents']):
    if cariIndex(value, '') != 0:
        index2.append(idx)

index = [value for value in index1 if value in index2]
rules.loc[index].sort_values(by=['lift'], ascending = False).head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
185,"(MELO, XAND)","(OMPZ, PARI)",0.001107,0.001107,0.000277,0.250000,225.750000,0.000276,1.331857
184,"(OMPZ, PARI)","(MELO, XAND)",0.001107,0.001107,0.000277,0.250000,225.750000,0.000276,1.331857
182,"(OMPZ, XAND, PARI)",(MELO),0.000277,0.034330,0.000277,1.000000,29.129032,0.000267,inf
124,"(ULSN, MUCT)",(PARI),0.000277,0.048173,0.000277,1.000000,20.758621,0.000264,inf
180,"(OMPZ, MELO, XAND)",(PARI),0.000277,0.048173,0.000277,1.000000,20.758621,0.000264,inf
108,"(GAVI, ULSN)",(PRVF),0.000277,0.052326,0.000277,1.000000,19.111111,0.000262,inf
112,"(IBUP, XAND)",(PRVF),0.000277,0.052326,0.000277,1.000000,19.111111,0.000262,inf
107,"(ULSN, PRVF)",(GAVI),0.000554,0.030177,0.000277,0.500000,16.568807,0.000260,1.939646
183,"(OMPZ, XAND)","(PARI, MELO)",0.063953,0.000277,0.000277,0.004329,15.636364,0.000259,1.004070
186,"(PARI, MELO)","(OMPZ, XAND)",0.000277,0.063953,0.000277,1.000000,15.636364,0.000259,inf
